In [37]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import numpy as np
import time

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score

The dataset referenced in the paper was not accessible. The site referenced is not active anymore. Lot of searching on the web, ended in finding a dataset whose shape is same as what is mentioned in the paper, with respect to number of 

In [2]:
#reading training and test set
train = pd.read_csv("KDDTrain.csv")
test = pd.read_csv("KDDTest.csv")

In [3]:
print('Dimensions of Training Set ',train.shape)
print('Dimensions of Test Set ',test.shape)

Dimensions of Training Set  (125973, 42)
Dimensions of Test Set  (22544, 42)


In [4]:
#first 5 records from training set
train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [5]:
df_train_str = train.select_dtypes(include=[object])

In [6]:
#print fields that in string format
str_fld_list = list(df_train_str.columns)
str_fld_list

['protocol_type', 'service', 'flag', 'class']

In [7]:
from sklearn.preprocessing import LabelEncoder

#encoding train
train_encode_dict = {}
train_conv_feat_dict = {}
for feat in str_fld_list:
    le = LabelEncoder()
    le.fit(df_train_str[feat])
    
    #storing encode model for future decode
    train_encode_dict[feat] = le
    train_conv_feat_dict[feat] = le.transform(df_train_str[feat])


for feat in str_fld_list:
    a = train_encode_dict[feat]
    print(a.classes_)

for feat in str_fld_list:
    a = train_conv_feat_dict[feat]
    print(a)   
    
    
#now adding the transformed field into the df

for feat in str_fld_list:
    del train[feat]
    train[feat] = train_conv_feat_dict[feat]



['icmp' 'tcp' 'udp']
['IRC' 'X11' 'Z39_50' 'aol' 'auth' 'bgp' 'courier' 'csnet_ns' 'ctf'
 'daytime' 'discard' 'domain' 'domain_u' 'echo' 'eco_i' 'ecr_i' 'efs'
 'exec' 'finger' 'ftp' 'ftp_data' 'gopher' 'harvest' 'hostnames' 'http'
 'http_2784' 'http_443' 'http_8001' 'imap4' 'iso_tsap' 'klogin' 'kshell'
 'ldap' 'link' 'login' 'mtp' 'name' 'netbios_dgm' 'netbios_ns'
 'netbios_ssn' 'netstat' 'nnsp' 'nntp' 'ntp_u' 'other' 'pm_dump' 'pop_2'
 'pop_3' 'printer' 'private' 'red_i' 'remote_job' 'rje' 'shell' 'smtp'
 'sql_net' 'ssh' 'sunrpc' 'supdup' 'systat' 'telnet' 'tftp_u' 'tim_i'
 'time' 'urh_i' 'urp_i' 'uucp' 'uucp_path' 'vmnet' 'whois']
['OTH' 'REJ' 'RSTO' 'RSTOS0' 'RSTR' 'S0' 'S1' 'S2' 'S3' 'SF' 'SH']
['anomaly' 'normal']
[1 2 1 ..., 1 1 1]
[20 44 49 ..., 54 30 20]
[9 9 5 ..., 9 5 9]
[1 1 0 ..., 1 0 1]


In [8]:
df_test_str = test.select_dtypes(include=[object])

#print fields that in string format
str_fld_list = list(df_test_str.columns)
str_fld_list

#encoding test
test_encode_dict = {}
test_conv_feat_dict = {}
for feat in str_fld_list:
    le = LabelEncoder()
    le.fit(df_test_str[feat])
    
    #storing encode model for future decode
    test_encode_dict[feat] = le
    test_conv_feat_dict[feat] = le.transform(df_test_str[feat])


for feat in str_fld_list:
    a = test_encode_dict[feat]
    print(a.classes_)

for feat in str_fld_list:
    a = test_conv_feat_dict[feat]
    print(a)   
    
    
#now adding the transformed field into the df

for feat in str_fld_list:
    del test[feat]
    test[feat] = test_conv_feat_dict[feat]



['icmp' 'tcp' 'udp']
['IRC' 'X11' 'Z39_50' 'auth' 'bgp' 'courier' 'csnet_ns' 'ctf' 'daytime'
 'discard' 'domain' 'domain_u' 'echo' 'eco_i' 'ecr_i' 'efs' 'exec' 'finger'
 'ftp' 'ftp_data' 'gopher' 'hostnames' 'http' 'http_443' 'imap4' 'iso_tsap'
 'klogin' 'kshell' 'ldap' 'link' 'login' 'mtp' 'name' 'netbios_dgm'
 'netbios_ns' 'netbios_ssn' 'netstat' 'nnsp' 'nntp' 'ntp_u' 'other'
 'pm_dump' 'pop_2' 'pop_3' 'printer' 'private' 'remote_job' 'rje' 'shell'
 'smtp' 'sql_net' 'ssh' 'sunrpc' 'supdup' 'systat' 'telnet' 'tftp_u'
 'tim_i' 'time' 'urp_i' 'uucp' 'uucp_path' 'vmnet' 'whois']
['OTH' 'REJ' 'RSTO' 'RSTOS0' 'RSTR' 'S0' 'S1' 'S2' 'S3' 'SF' 'SH']
['anomaly' 'normal']
[1 1 1 ..., 1 2 1]
[45 45 19 ..., 22 11 52]
[1 1 9 ..., 9 9 1]
[0 0 1 ..., 0 1 0]


In [10]:
#transformed data
train.head()


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type,service,flag,class
0,0,491,0,0,0,0,0,0,0,0,...,0.17,0.00,0.00,0.00,0.05,0.00,1,20,9,1
1,0,146,0,0,0,0,0,0,0,0,...,0.88,0.00,0.00,0.00,0.00,0.00,2,44,9,1
2,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,1.00,1.00,0.00,0.00,1,49,5,0
3,0,232,8153,0,0,0,0,0,1,0,...,0.03,0.04,0.03,0.01,0.00,0.01,1,24,9,1
4,0,199,420,0,0,0,0,0,1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,1,24,9,1


In [11]:
test.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type,service,flag,class
0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,1.00,1.00,1,45,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,1.00,1.00,1,45,1,0
2,2,12983,0,0,0,0,0,0,0,0,...,0.61,0.02,0.0,0.0,0.00,0.00,1,19,9,1
3,0,20,0,0,0,0,0,0,0,0,...,1.00,0.28,0.0,0.0,0.00,0.00,0,13,9,0
4,1,0,15,0,0,0,0,0,0,0,...,0.03,0.02,0.0,0.0,0.83,0.71,1,55,2,0


In [19]:
#splitting df into features and predicted value
y_train = train['class']
X_train = train.copy()
del X_train['class']


y_test = test['class']
X_test = test.copy()
del X_test['class']

In [20]:
logistic = linear_model.LogisticRegression()
logistic.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
#accuracy
sum(logistic.predict(X_test)==y_test)/y_test.shape[0]

0.68355216465578428

In [32]:
from sklearn.grid_search import GridSearchCV

start_time = time.clock()
clfl2=linear_model.LogisticRegression()
parameters = {"C": [0.0001, 0.001, 0.1, 1, 10, 100]}
fitmodel = GridSearchCV(clfl2, param_grid=parameters, cv=10, scoring="accuracy")
fitmodel.fit(X_train, y_train)
print(time.clock() - start_time, "seconds")
fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_, fitmodel.grid_scores_


(LogisticRegression(C=0.0001, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 {'C': 0.0001},
 0.87737054765703759,
 [mean: 0.87737, std: 0.00789, params: {'C': 0.0001},
  mean: 0.87529, std: 0.00593, params: {'C': 0.001},
  mean: 0.87581, std: 0.00792, params: {'C': 0.1},
  mean: 0.87293, std: 0.00461, params: {'C': 1},
  mean: 0.87493, std: 0.00692, params: {'C': 10},
  mean: 0.87357, std: 0.00574, params: {'C': 100}])

In [39]:
start_time = time.clock()
clfl2=linear_model.LogisticRegression(C=fitmodel.best_params_['C'])
clfl2.fit(X_train, y_train)
ypred2=clfl2.predict(X_test)
print(accuracy_score(ypred2, y_test))
print(time.clock() - start_time, "seconds")

0.68390702626
1.6067370000000096 seconds


### Decision Trees

In [22]:
from sklearn import tree
part = tree.DecisionTreeClassifier()
part.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [23]:
#accuracy
sum(part.predict(X_test)==y_test)/y_test.shape[0]

0.7737757274662882

In [44]:
start_time = time.clock()
clfl3 = tree.DecisionTreeClassifier()
parameters = {"criterion": ['gini','entropy'],'min_samples_split':[2,10,50,100],'splitter':['best','random']}
fitmodel = GridSearchCV(clfl3, param_grid=parameters, cv=10, scoring="accuracy")
fitmodel.fit(X_train, y_train)
print(fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_, fitmodel.grid_scores_)
print(time.clock() - start_time, "seconds")

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best') {'min_samples_split': 2, 'splitter': 'best', 'criterion': 'entropy'} 0.998348852532 [mean: 0.99814, std: 0.00052, params: {'min_samples_split': 2, 'splitter': 'best', 'criterion': 'gini'}, mean: 0.99760, std: 0.00031, params: {'min_samples_split': 2, 'splitter': 'random', 'criterion': 'gini'}, mean: 0.99799, std: 0.00044, params: {'min_samples_split': 10, 'splitter': 'best', 'criterion': 'gini'}, mean: 0.99702, std: 0.00070, params: {'min_samples_split': 10, 'splitter': 'random', 'criterion': 'gini'}, mean: 0.99743, std: 0.00027, params: {'min_samples_split': 50, 'splitter': 'best', 'criterion': 'gini'}, mean: 0.99559, std: 0.00059, params: {'min_samples_split': 50, 'splitter': 'random', 'criterion': 'gini'}, mean

In [45]:
start_time = time.clock()
clfl3=tree.DecisionTreeClassifier(criterion=fitmodel.best_params_['criterion'],min_samples_split=fitmodel.best_params_['min_samples_split'],splitter=fitmodel.best_params_['splitter'])
clfl3.fit(X_train, y_train)
ypred3=clfl3.predict(X_test)
print(accuracy_score(ypred3, y_test))
print(time.clock() - start_time, "seconds")

0.804648687012
0.5379780000000096 seconds


### Naive Bayes

In [24]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [25]:
#accuracy
sum(gnb.predict(X_test)==y_test)/y_test.shape[0]

0.45031937544357703

In [47]:
start_time = time.clock()
clfl4 = GaussianNB()
parameters = {}
fitmodel = GridSearchCV(clfl4, param_grid=parameters, cv=10, scoring="accuracy")
fitmodel.fit(X_train, y_train)
print(fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_, fitmodel.grid_scores_)
print(time.clock() - start_time, "seconds")

GaussianNB() {} 0.534193835187 [mean: 0.53419, std: 0.00097, params: {}]
2.4734129999999936 seconds


In [48]:
start_time = time.clock()
clfl4= GaussianNB()
clfl4.fit(X_train, y_train)
ypred4=clfl4.predict(X_test)
print(accuracy_score(ypred4, y_test))
print(time.clock() - start_time, "seconds")

0.450319375444
0.17779100000001336 seconds


### Random Forest

In [26]:
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier(n_estimators=25)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
#accuracy
sum(rf.predict(X_test)==y_test)/y_test.shape[0]

0.76871894960965226

In [53]:
RandomForestClassifier?

In [54]:
start_time = time.clock()
clfl5 = RandomForestClassifier()
parameters = {"n_estimators": [5,10,20,25,50,100],'criterion':['gini','entropy'],'min_samples_split':[2,10,50,100]}
fitmodel = GridSearchCV(clfl5, param_grid=parameters, cv=10, scoring="accuracy")
fitmodel.fit(X_train, y_train)
print(fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_, fitmodel.grid_scores_)
print(time.clock() - start_time, "seconds")

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False) {'min_samples_split': 2, 'n_estimators': 100, 'criterion': 'entropy'} 0.999015662086 [mean: 0.99865, std: 0.00031, params: {'min_samples_split': 2, 'n_estimators': 5, 'criterion': 'gini'}, mean: 0.99878, std: 0.00019, params: {'min_samples_split': 2, 'n_estimators': 10, 'criterion': 'gini'}, mean: 0.99895, std: 0.00024, params: {'min_samples_split': 2, 'n_estimators': 20, 'criterion': 'gini'}, mean: 0.99883, std: 0.00027, params: {'min_samples_split': 2, 'n_estimators': 25, 'criterion': 'gini'}, mean: 0.99899, std: 0.00022, params: {'min_samples_split': 2, 'n_estimators': 50, 'criterion': 'gini'}, mean: 0.99900, std: 0.00028, param

In [56]:
start_time = time.clock()
clfl5=RandomForestClassifier(criterion=fitmodel.best_params_['criterion'],min_samples_split=fitmodel.best_params_['min_samples_split'],n_estimators=fitmodel.best_params_['n_estimators'])
clfl5.fit(X_train, y_train)
ypred5=clfl5.predict(X_test)
print(accuracy_score(ypred5, y_test))
print(time.clock() - start_time, "seconds")

0.775993612491
7.918926000000056 seconds


In [ ]:
#adabost
#ROC Curve